In [ ]:
!wget https://gist.githubusercontent.com/johnidm/1cb74e2603177d3d3d554b6b1fe79728/raw/49e83c8178a16a76518894af8c00c2f4fb80a171/dataset.jsonl

In [ ]:
import json


with open('dataset.jsonl', 'r') as f:
    lines = list(f)

training_data: list = []

for line in lines:
    break
    row = json.loads(line)
    training_data.append(  [ row["text"], { "entities": row["label"] } ] )
print(len(training_data))

In [ ]:
train_split = int(len(training_data) * 0.8) # 80% training and 20% deve set

train_data = training_data[:train_split]
dev_data = training_data[train_split:]


In [11]:
print(train_data[0][1])

{'entities': [[419, 498, 'AFFILIATIONS']]}


In [ ]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm


def convert(path, dataset):
    nlp = spacy.blank("en")
    db = DocBin()
    for text, annot in tqdm(dataset): 
            doc = nlp.make_doc(text) 
            ents = []
            for start, end, label in annot["entities"]:
                span = doc.char_span(start, end, label=label, alignment_mode="contract")
                if span is None:
                    print("Skipping nil entity")                
                if span.text != span.text.strip():
                    print("Skipping entity spans with whitespace")
                else:
                    ents.append(span)
            doc.ents = ents

            db.add(doc)
    db.to_disk(path)
    
convert("train.spacy", train_data)
convert("dev.spacy", dev_data)